## VT Workbook - VBA Analysis

This workbook shows a simple example of analysing a family of malware using the VT API. To use this you will need a VirusTotal API key which needs to be set below.

This will allow us to download a set of VBA samples with a low detection rate, and extract the macros contained within each.

**Due to the API key being used, I'd recommend that this isn't run outside of your trusted environment**

In [ ]:
%load_ext dotenv
%dotenv

API_KEY = "API_KEY_HERE"
VT_LIMIT = 10
VT_SEARCH = "tag:powershell fs:20d- type:doc positives:5-"
TARGET_DIR = "/tmp/samples"

# Loaded via .env
if os.getenv('API_KEY') != None:
    API_KEY = os.getenv('API_KEY')

First step we need to gather some samples to analyse. We will do this via the VT API, downloading samples to the `TARGET_DIR` directory for later analysis (also means we can repeat analysis without using up VT API call credits ;)

In [ ]:
import sys
import nest_asyncio
import ipywidgets as widgets
import pandas as pd

nest_asyncio.apply()
sys.path.append('../src/')

from helpers import vthelper, vba

results = []

# Retrieve search results from VT and download each sample to TARGET_DIR
vt = vthelper.VirusTotal(API_KEY, limit=VT_LIMIT)
for result in vt.downloadSearchResults(VT_SEARCH, TARGET_DIR):
    results.append((result.sha256, result.meaningful_name, result.tags))
    
# Display our results
pd.set_option('max_colwidth', 800)
df = pd.DataFrame(data=results, columns=["Hash","Name","Tags"])
display(df)

# results contains 3 elements:
# [0] - SHA256 hash
# [1] - Name
# [2] - VT Tags

In [ ]:
!ls -alF {TARGET_DIR}

With our samples downloaded, we can now analyse each VBA project using oletools. Each identified VBA sample will be displayed.

In [ ]:
tabs = []
for result in results:
    tabs.append(widgets.Output())
    with tabs[-1]:
        analysis = vba.VBAAnalysis(f"{TARGET_DIR}/{result[0]}")
        for code in analysis.analyse():
            print(f"' SAMPLE HASH: {result[0]}")
            print(code)

out = widgets.Tab(tabs)
for i in range(len(results)):
    out.set_title(i, results[i][1])

display(out)

In [ ]:
stored = []
for i in range(len(results)):
    stored.append({"path": f"{TARGET_DIR}/{results[i][0]}", "name": results[i][1], "hash": results[i][0]})

%store stored